In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/DATN5T/

/content/drive/MyDrive/DATN5T


In [ ]:
pip install --quiet transformers==4.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 59.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 101.9 MB/s eta 0:00:00


In [ ]:
pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.4 MB/s eta 0:00:00


In [ ]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)
from tqdm.auto import tqdm

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("hotai/T5-small-vi-sum")

In [ ]:
import torch

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import transformers
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap

In [ ]:
import pandas as pd

In [ ]:
test_df=pd.read_csv('DataTest_v4.csv')
test_df = test_df.loc[:, ~test_df.columns.str.contains('^Unnamed')]

train_df=pd.read_csv('DataTrain_v4.csv')
train_df = train_df.loc[:, ~train_df.columns.str.contains('^Unnamed')]

val_df=pd.read_csv('DataVal_v4.csv')
val_df = val_df.loc[:, ~val_df.columns.str.contains('^Unnamed')]

In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(style='whitegrid', palette='muted', font_scale = 1.2)
rcParams['figure.figsize'] = 16,10

In [ ]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [ ]:
class NewsSummaryDataset(Dataset):
 def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: T5Tokenizer,
      text_max_token_len: int = 512,
      summary_max_token_len: int = 128
 ):
   self.tokenizer = tokenizer
   self.data = data
   self.text_max_token_len = text_max_token_len
   self.summary_max_token_len = summary_max_token_len
 def __len__(self):
   return len(self.data)
 def __getitem__(self, index : int):
   data_row = self.data.iloc[index]

   text = data_row["Content"]
   text_encoding = tokenizer(
       text,
       max_length = self.text_max_token_len,
       padding = "max_length",
       truncation = True,
       return_attention_mask = True,
       add_special_tokens = True,
       return_tensors = "pt"
   )
   summary_encoding = tokenizer(
       data_row["Summary"],
       max_length = self.summary_max_token_len,
       padding = "max_length",
       truncation = True,
       return_attention_mask = True,
       add_special_tokens = True,
       return_tensors = "pt"
   )
   labels = summary_encoding["input_ids"]
   labels[labels == 0] = -100

   return dict(
       text = text,
       summary = data_row["Summary"],
       text_input_ids = text_encoding["input_ids"].flatten(),
       text_attention_mask = text_encoding["attention_mask"].flatten(),
       labels = labels.flatten(),
       labels_attention_mask = summary_encoding["attention_mask"].flatten()
   )

In [ ]:
class NewsSummaryDataModule(pl.LightningDataModule):
   def __init__(
       self,
       train_df: pd.DataFrame,
       test_df: pd.DataFrame,
       tokenizer: T5Tokenizer,
       batch_size: int = 8 ,
       text_max_token_len: int =512,
       summary_max_token_len: int = 128
   ):
       super().__init__()
       self.train_df = train_df
       self.test_df = test_df

       self.batch_size = batch_size
       self.tokenizer = tokenizer

       self.text_max_token_len = text_max_token_len
       self.summary_max_token_len = summary_max_token_len
   def setup(self, stage = None):
      self.train_dataset = NewsSummaryDataset(
          self.train_df,
          self.tokenizer,
          self.text_max_token_len,
          self.summary_max_token_len
      )
      self.test_dataset = NewsSummaryDataset(
          self.test_df,
          self.tokenizer,
          self.text_max_token_len,
          self.summary_max_token_len
      )
   def train_dataloader(self):
      return DataLoader(
          self.train_dataset,
          batch_size = self.batch_size,
          shuffle = True,
          num_workers = 2
      )
   def val_dataloader(self):
     return DataLoader(
       self.test_dataset,
       batch_size = self.batch_size,
       shuffle = True,
       num_workers = 2
     )

In [ ]:
num_epochs = 10
BATCH_SIZE = 32

data_module = NewsSummaryDataModule(train_df, test_df, tokenizer,batch_size = BATCH_SIZE)

In [ ]:
class NewsSummaryModel(pl.LightningModule):
   def __init__(self):
     super().__init__()
     self.model =  T5ForConditionalGeneration.from_pretrained("hotai/T5-small-vi-sum", return_dict = True)
   def forward(self, input_ids, attention_mask, decoder_attention_mask, labels = None):
     output = self.model(
         input_ids,
         attention_mask = attention_mask,
         labels = labels,
         decoder_attention_mask = decoder_attention_mask
     )

     return output.loss, output.logits

   def training_step(self, batch, batch_idx):
     input_ids = batch["text_input_ids"]
     attention_mask = batch["text_attention_mask"]
     labels = batch["labels"]
     labels_attention_mask = batch["labels_attention_mask"]

     loss, outputs = self(
         input_ids = input_ids,
         attention_mask = attention_mask,
         decoder_attention_mask = labels_attention_mask,
         labels = labels
     )
     self.log("train_loss", loss, prog_bar = True, logger = True)
     return loss
   def validation_step(self, batch, batch_idx):
     input_ids = batch["text_input_ids"]
     attention_mask = batch["text_attention_mask"]
     labels = batch["labels"]
     labels_attention_mask = batch["labels_attention_mask"]

     loss, outputs = self(
         input_ids = input_ids,
         attention_mask = attention_mask,
         decoder_attention_mask = labels_attention_mask,
         labels = labels
     )
     self.log("val_loss", loss, prog_bar = True, logger = True)
     return loss

   def test_step(self, batch, batch_idx):
     input_ids = batch["text_input_ids"]
     attention_mask = batch["text_attention_mask"]
     labels = batch["labels"]
     labels_attention_mask = batch["labels_attention_mask"]

     loss, outputs = self(
         input_ids = input_ids,
         attention_mask = attention_mask,
         decoder_attention_mask = labels_attention_mask,
         labels = labels
     )

     self.log("test_loss", loss, prog_bar = True, logger = True)
     return loss

   def configure_optimizers(self):
     return AdamW(self.parameters(), lr = 0.0001)

In [ ]:
model = NewsSummaryModel()

Downloading:   0%|          | 0.00/784 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
checkpoint_callback = ModelCheckpoint(
        dirpath ="./checkpoints",
        filename = "best-checkpoint",
        save_top_k=1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )
logger = TensorBoardLogger("lightning_logs", name = "news-summary")

In [ ]:
import tensorflow as tf

In [ ]:
trainer = pl.Trainer(
  logger = logger ,
  callbacks  = checkpoint_callback,
  max_epochs = num_epochs,
  accelerator="gpu",
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
from keras.callbacks import History
history = History()

In [ ]:
import pickle

In [ ]:
history=trainer.fit(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /content/drive/MyDrive/DATN5T/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLo

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 15048: 'val_loss' reached 1.27858 (best 1.27858), saving model to '/content/drive/MyDrive/DATN5T/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 16929: 'val_loss' reached 1.26637 (best 1.26637), saving model to '/content/drive/MyDrive/DATN5T/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 18810: 'val_loss' reached 1.25391 (best 1.25391), saving model to '/content/drive/MyDrive/DATN5T/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
trained_model = NewsSummaryModel.load_from_checkpoint(
    'checkpoints/best-checkpoint.ckpt'
)

In [ ]:
trained_model.freeze()

In [ ]:
trained_model.to('cpu')

NewsSummaryModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(250112, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(250112, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedGeluDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): L

In [ ]:
def summarize(text):
  text_encoding = tokenizer(
      text,
      max_length = 512,
      padding = "max_length",
      truncation = True,
      return_attention_mask = True,
      add_special_tokens = True,
      return_tensors = "pt"
  )

  generated_ids = trained_model.model.generate(
      input_ids = text_encoding["input_ids"],
      attention_mask = text_encoding["attention_mask"],
      max_length = 150,
      num_beams = 2,
      repetition_penalty=2.5,
      length_penalty = 1.0,
      early_stopping = True
  )

  preds = [
      tokenizer.decode(gen_id, skip_special_tokens = True,clean_up_tokenization_spaces= True )
      for gen_id in generated_ids
  ]

  return "".join(preds)

In [ ]:
sample_row = test_df.iloc[0]
text = sample_row["Content"]

In [ ]:
text

'Sáng 23.8 , \xa0Chủ tịch Nước Trần Đại Quang\xa0và Phu nhân rời Hà Nội , lên đường đi thăm cấp Nhà nước đến Cộng hoà Dân chủ Liên bang Ethiopia từ ngày 23-25.8 , theo lời mời của Tổng thống Mulatu Teshome . Đây là chuyến thăm đầu tiên của người đứng đầu Nhà nước Việt Nam đến Ethiopia kể từ khi hai nước thiết lập quan hệ ngoại giao năm 1976 , là dấu mốc lịch sử trong quan hệ Việt Nam-Ethiopia , mở ra triển vọng hợp tác đầu tư và thương mại giữa hai nước trong những năm tới . Ethiopia là một trong những quốc gia lâu đời nhất ở Châu Phi , có tiềm năng phát triển cả về nông nghiệp và công nghiệp . Nông nghiệp chiếm vị trí hàng đầu trong nền kinh tế quốc dân Ethiopia 85% lao động , 90% tổng thu xuất nhập khẩu , 46% tổng sản phẩm nội địa , nhưng cho đến nay , chỉ có hơn 10% trong tổng số 790.000 km2 đất nông nghiệp được khai thác - theo TTXVN . Giữa năm 2015 , Chính phủ Ethiopia đã thông qua Kế hoạch phát triển Tăng trưởng và Chuyển đổi lần thứ 2 , ưu tiên công nghiệp hoá , đô thị hoá và đầ

In [ ]:
sample_row["Summary"]

'Chuyến thăm là dịp để lãnh đạo Việt Nam và các nước Ethiopia và Ai Cập trao đổi các biện pháp thúc đẩy hơn nữa quan hệ hợp tác trên các lĩnh vực kinh tế , thương mại , đầu tư tìm kiếm cơ hội hợp tác trong các lĩnh vực có thế mạnh , như nông nghiệp , văn hoá , du lịch .'

In [ ]:
model_summary = summarize(text)

In [ ]:
model_summary

'Chủ tịch Nước Trần Đại Quang và Phu nhân rời Hà Nội, lên đường đi thăm cấp Nhà nước đến Cộng hoà Dân chủ Liên bang Ethiopia.'

In [ ]:
checkpoint_callback = ModelCheckpoint(
        dirpath ="./checkpoints",
        filename = "best-checkpoint_cpu",
        save_top_k=1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )
logger = TensorBoardLogger("lightning_logs", name = "news-summary")

In [ ]:
trainer = pl.Trainer(
  logger = logger ,
  callbacks  = checkpoint_callback,
  max_epochs = 0,
  accelerator="cpu",
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [ ]:
trainer.fit(trained_model, data_module)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 300 M 
-----------------------------------------------------
0         Trainable params
300 M     Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=0` reached.


In [ ]:
trainer.save_checkpoint("./best-checkpoint_cpu.ckpt")

In [ ]:
trained_model_v1 = NewsSummaryModel.load_from_checkpoint(
    './best-checkpoint_cpu.ckpt'
)

In [ ]:
trained_model.freeze()

In [ ]:
trained_model.to('cpu')

NewsSummaryModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(250112, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(250112, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedGeluDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): L

In [ ]:
sample_row = test_df.iloc[2]
text = sample_row["Content"]
model_summary = summarize(text)

In [ ]:
text

'Họ rất thích cảnh đẹp Vịnh Rồng nhưng chê thậm tệ môi trường bụi lửng và rác . Hạ Long mênh mông là rác , họ nói và cho biết không muốn trở lại nữa . Khi tôi muốn hỏi rõ hơn , họ chất vấn ngược tại sao vịnh đẹp như thế , phí dịch vụ đắt thế mà môi trường lại ngập rác vì sao khách phải xếp hàng rất lâu trong khi hoàn toàn có thể triển khai bán vé điện tử ... Họ thậm chí nhìn ra tình trạng tham nhũng vặt ở các điểm tham quan . Lăn lộn nhiều năm quảng bá và thu hút khách quốc tế , chúng tôi từng phát động chiến dịch WOW Vietnam , dựa trên những thế mạnh của đất nước để đưa ra bảy lý do đặt theo các ký tự đầu của từ Việt Nam mời gọi du khách tới thăm , gồm V - Varied landscape Phong cảnh đẹp , I - Indigenous culture Văn hoá bản địa đặc sắc , E - Exotic beaches Bãi biển đẹp , T - Timeless charm Vẻ quyến rũ vô tận , N - Natural heritage sites Các di sản thiên nhiên , A - Ancient cities Các đô thị cổ , M - Memories to cherish forever Trải nghiệm đáng nhớ . Nhưng tôi cũng không lạ lẫm gì với 

In [ ]:
sample_row["Summary"]

'Tôi vừa tiễn một đoàn khách Pháp 10 người , đi du thuyền Hạ Long tham quan hai ngày một đêm .'

In [ ]:
model_summary

'Từng phát động chiến dịch WOW Vietnam, Hạ Long mênh mông là rác nhưng chê thậm tệ môi trường bụi lửng.'